In [1]:
import pdfplumber
import re
import json
import os
import html
import PyPDF2
import pandas as pd


from dotenv import load_dotenv
from neo4j import GraphDatabase
from py2neo import Graph, Node, Relationship
from langchain_openai import AzureChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain.text_splitter import RecursiveCharacterTextSplitter
from typing import List, Dict, Tuple

## METODI

In [2]:
load_dotenv()
# Recupera endpoint e chiave dalle variabili d'ambiente
api_base = os.getenv("AZURE_OPENAI_API_BASE")
api_key = os.getenv("AZURE_OPENAI_API_KEY")

# neo4j first instance parameters
NEO4J_URI= "neo4j+s://0482640f.databases.neo4j.io"
NEO4J_USERNAME= "neo4j"
NEO4J_PASSWORD= "PNvdaZlk326-ja2hRD1K97ZUUMnD4mj0NsecZNu5-9k"
AURA_INSTANCEID= "0482640f"
AURA_INSTANCENAME= "Instance01"

driver = GraphDatabase.driver(NEO4J_URI, auth=(NEO4J_USERNAME, NEO4J_PASSWORD))

In [3]:
def crea_llm(api_key: str, api_base: str) -> AzureChatOpenAI:
    """
    Crea un'istanza del modello linguistico Azure OpenAI.
    
    Args:
        api_key (str): La chiave API di Azure
        api_base (str): L'URL base dell'API Azure
        
    Returns:
        AzureChatOpenAI: Istanza configurata del modello
    """
    return AzureChatOpenAI(
        openai_api_version="2024-08-01-preview",
        azure_deployment="o1-mini",
        azure_endpoint=api_base,
        api_key=api_key,
        temperature=1
    )

llm = crea_llm(api_key, api_base)

### METODI

In [ ]:
def carica_pianeti_e_distanze(file_csv: str):
    """
    Carica i pianeti come nodi e le distanze tra di loro come relazioni nel database Neo4j.
    
    Args:
        file_csv (str): Percorso del file CSV contenente le distanze tra i pianeti.
        uri (str): URI del database Neo4j.
        user (str): Nome utente per Neo4j.
        password (str): Password per Neo4j.
    """
    
    # Carica il file CSV usando pandas
    df = pd.read_csv(file_csv, index_col=0)
    
    with driver.session() as session:
        # Crea i nodi per i pianeti
        pianeti = df.columns.tolist()  # Estrae i nomi dei pianeti
        
        for pianeta in pianeti:
            # Crea un nodo per ciascun pianeta
            query = "MERGE (p:Pianeta {nome: $nome})"
            session.run(query, nome=pianeta)
        
        # Crea le relazioni DISTANZA tra i pianeti
        for i, pianeta1 in enumerate(pianeti):
            for j, pianeta2 in enumerate(pianeti):
                if i < j:  # Evita di creare relazioni duplicate (ad esempio tra Tatooine e Asgard e viceversa)
                    distanza = df.at[pianeta1, pianeta2]
                    # Crea una relazione con la distanza tra i pianeti
                    query = """
                    MATCH (p1:Pianeta {nome: $pianeta1}), (p2:Pianeta {nome: $pianeta2})
                    MERGE (p1)-[:DISTANZA {km: $distanza}]->(p2)
                    """
                    session.run(query, pianeta1=pianeta1, pianeta2=pianeta2, distanza=distanza)
        
        print("Pianeti e distanze caricate correttamente in Neo4j.")

In [ ]:

def carica_piatti(file_json: str):
    """
    Carica i piatti e i numeri associati come nodi nel database Neo4j.
    
    Args:
        file_json (str): Percorso del file JSON contenente i piatti e i numeri.
        uri (str): URI del database Neo4j.
        user (str): Nome utente per Neo4j.
        password (str): Password per Neo4j.
    """
    
    # Carica il file JSON
    with open(file_json, 'r', encoding='utf-8') as f:
        piatti_data = json.load(f)
    
    with driver.session() as session:
        for piatto, numero in piatti_data.items():
            # Decodifica i caratteri speciali (ad esempio per trattare le sequenze \u00e0)
            piatto_decodificato = html.unescape(piatto)
            
            # Crea un nodo per ciascun piatto
            query = """
            MERGE (p:Piatto {nome: $nome})
            SET p.numero = $numero
            """
            session.run(query, nome=piatto_decodificato, numero=numero)
        
        print("Piatti caricati correttamente in Neo4j.")

## MAIN

In [ ]:
# Esegui il codice passando il percorso del file CSV e le credenziali di Neo4j
file_csv = '../Hackapizza Dataset/Misc/Distanze.csv'  # Percorso al tuo file CSV

carica_pianeti_e_distanze(file_csv)


Pianeti e distanze caricate correttamente in Neo4j.


In [ ]:
# Esegui il codice passando il percorso del file JSON e le credenziali di Neo4j
file_json = '../Hackapizza Dataset/Misc/dish_mapping.json'  # Percorso al tuo file JSON

carica_piatti(file_json)


Piatti caricati correttamente in Neo4j.
